# Use MACS3 API to call peaks for each cluster in scATAC-seq data

The scATAC-seq data was downloaded from [10X genomics website](https://www.10xgenomics.com/datasets/10k-human-pbmcs-atac-v2-chromium-x-2-standard). The files used in this tutorial are:
- [Clustering analysis](https://cf.10xgenomics.com/samples/cell-atac/2.1.0/10k_pbmc_ATACv2_nextgem_Chromium_X/10k_pbmc_ATACv2_nextgem_Chromium_X_analysis.tar.gz) for the clustering results
- [Fragments](https://cf.10xgenomics.com/samples/cell-atac/2.1.0/10k_pbmc_ATACv2_nextgem_Chromium_X/10k_pbmc_ATACv2_nextgem_Chromium_X_fragments.tsv.gz) for the locations of aligned fragments in TSV format
- [Per Barcode Metrics](https://cf.10xgenomics.com/samples/cell-atac/2.1.0/10k_pbmc_ATACv2_nextgem_Chromium_X/10k_pbmc_ATACv2_nextgem_Chromium_X_singlecell.csv) for the information of each barcode

## Load the fragment file and build pileup track

We will import the fragment file parser from MACS3.

In [1]:
# %pip install macs3 panda

from MACS3.IO.Parser import FragParser
from MACS3.IO.BedGraphIO import bedGraphIO
from MACS3.Utilities.Logger import logging

import numpy as np
import warnings

np.seterr(all='raise')
warnings.filterwarnings("error", category=RuntimeWarning)

# we can use the pre-configured logging function from MACS3 to monitor memory usage...
logger = logging.getLogger("demo")
info = logger.info
# Note, you can replace `print` with `info` to show the time and memory usage at that moment.


The fragment file is `10k_pbmc_ATACv2_nextgem_Chromium_X_fragments.tsv.gz`. Note that MACS3 can directly load gzipped files. We will load the file and build a `PairEndTrack.PETrackII` object, which contains alignment locations, barcodes and counts.

In [2]:
frag_file = FragParser("./10k_pbmc_ATACv2_nextgem_Chromium_X_fragments.tsv.gz", buffer_size=100000)
petrack = frag_file.build_petrack(max_count=2)
petrack.finalize()

INFO  @ 15 Nov 2025 10:52:09: [196 MB]  1000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:09: [237 MB]  2000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:10: [257 MB]  3000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:11: [281 MB]  4000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:11: [285 MB]  5000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:12: [299 MB]  6000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:12: [310 MB]  7000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:13: [310 MB]  8000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:14: [310 MB]  9000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:14: [310 MB]  10000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:15: [328 MB]  11000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:16: [344 MB]  12000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:16: [359 MB]  13000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:17: [373 MB]  14000000 fragments parsed 
INFO  @ 15 Nov 2025 10:52:18: [399 MB]  15000000 fragments parsed 
INFO

Before we call peaks, let's clean up the data by removing uncommon chromosomes from downstream analysis. Let's check first...

In [3]:
petrack_rlengths = petrack.rlengths
print(list(petrack_rlengths.keys()))

[b'GL000009.2', b'GL000194.1', b'GL000195.1', b'GL000205.2', b'GL000213.1', b'GL000218.1', b'GL000219.1', b'KI270711.1', b'KI270713.1', b'KI270721.1', b'KI270726.1', b'KI270727.1', b'KI270728.1', b'KI270731.1', b'KI270734.1', b'chr1', b'chr10', b'chr11', b'chr12', b'chr13', b'chr14', b'chr15', b'chr16', b'chr17', b'chr18', b'chr19', b'chr2', b'chr20', b'chr21', b'chr22', b'chr3', b'chr4', b'chr5', b'chr6', b'chr7', b'chr8', b'chr9', b'chrX', b'chrY']


Let's only keep chr1 to chrY. In order to do so, we need to construct genomic 'regions' covering the uncommon chromosomes, and then 'exclude' any fragments overlapping with those regions.

In [4]:
from MACS3.Signal.Region import Regions

regions_to_be_excluded = Regions()
for chrom, chrom_length in petrack_rlengths.items():
    if not chrom.startswith(b'chr'): # this rule may not work if you want to exclude chomosome such as chr1_random, so adjust it if it's necessary
        regions_to_be_excluded.add_loc(chrom, 0, chrom_length)

# then use .exclude of PETrackII 
petrack.exclude(regions_to_be_excluded)
petrack.finalize()

Now check the chromosome names again.

In [5]:
petrack_rlengths = petrack.rlengths
print(list(petrack_rlengths.keys()))

[b'chr1', b'chr10', b'chr11', b'chr12', b'chr13', b'chr14', b'chr15', b'chr16', b'chr17', b'chr18', b'chr19', b'chr2', b'chr20', b'chr21', b'chr22', b'chr3', b'chr4', b'chr5', b'chr6', b'chr7', b'chr8', b'chr9', b'chrX', b'chrY']


Now we can check some basic statistics of the loaded data.

In [6]:
print(f"average template length is {petrack.average_template_length}")
print(f"total number of fragments is {petrack.total}")

average template length is 153.16387939453125
total number of fragments is 331064421


## Call peaks for the entire dataset

Next, we will call peaks for the entire dataset. The first step is to build a pileup track from the alignments. But before that, we can filter out invalid cells/barcodes. For example, we have an `10k_pbmc_ATACv2_nextgem_Chromium_X_singlecell.csv` file downloaded together with the fragment file which is from the standard 10x pipeline. If we want to only keep the cell barcodes with at least 500 usable fragments and at least 25% of total fragments are marked as usable:

In [7]:
import pandas as pd

barcodes = []

# Read the CSV
df = pd.read_csv("./10k_pbmc_ATACv2_nextgem_Chromium_X_singlecell.csv")

# We keep the cell barcode that 1) is a cell barcode 2) more than 500 usable fragments 3) at least 25% of total fragments are usable
df_pass = df[
    (df["is__cell_barcode"] == 1) & 
    (df["passed_filters"] >= 500) &
    (df["passed_filters"]/df["total"] >= 0.25)]

# Extract barcode values as a list then convert to a set
# Note: we need to encode the string since PETrackII.subset needs bytestrings.
barcodes = set([x.encode() for x in df_pass["barcode"].tolist()])

print("Cell barcodes to be kept:", len(barcodes))

petrack = petrack.subset(barcodes)

Cell barcodes to be kept: 10270


Now we can build the pileup signal track of all valid cell barcodes and their fragments.

In [8]:
pileup_track = petrack.pileup_bdg()

We can get the sum, total length, maximum, minimum, mean, and standard deviation from the pileup track by using `summary` function.

In [9]:
(pileup_sum, pileup_length, pileup_max, pileup_min, pileup_mean, pileup_std) = pileup_track.summary()
print(f"{pileup_sum=}\n{pileup_length=}\n{pileup_max=}\n{pileup_min=}\n{pileup_mean=}\n{pileup_std=}")

pileup_sum=29146726400.0
pileup_length=3086966835
pileup_max=7802.0
pileup_min=0.0
pileup_mean=9.441865921020508
pileup_std=97.03312683105469


Next, we will call peaks for the entire dataset. Since we are calling peaks for ATAC-seq data, we will use the single whole-genome average pileup value as the background. From above calculation, the average is in the variable `pileup_mean`.

In [10]:
global_bg_track = pileup_track.set_single_value(pileup_mean) # this will return a new track with all values set as `pileup_mean`

We construct a score track for comparing observed pileup and background. The score track will contain the observed pileup, the background, and scores for each position in the genome.

In [11]:
score_track = pileup_track.make_ScoreTrackII_for_macs(global_bg_track, depth1=100, depth2=100) # Note, depth1 and 2 are the same so there is no need for scaling the values

We will use p-score (-log10 p-value) as scores. MACS3 supports the following methods:

- p: -log10 pvalue;
- q: -log10 qvalue;
- l: log10 likelihood ratio (minus for depletion)
- s: symmetric log10 likelihood ratio (for comparing two ChIPs)
- f: log10 fold enrichment
- F: linear fold enrichment
- d: subtraction
- M: maximum
- m: fragment pileup per million reads

To use any of the methods, provide the argument to `change_score_method` with `ord`, as shown in the following example.

In [12]:
score_track.change_score_method(ord('p'))

Then we can call peaks based on these newly generated score track. We will use `p-value`=0.00001 as cutoff, which will be translated into p-score cutoff of 5.

In [13]:
peaks = score_track.call_peaks(cutoff=5)

We can check the number of peaks and the total length of peaks by converting peaks to regions.

In [14]:
# make Regions for peaks from all cell barcodes
regions = Regions()
regions.init_from_PeakIO(peaks)
print(f"Number of peaks: {regions.total}")
print(f"Total basepairs of peaks: {regions.total_length()}")


Number of peaks: 254367
Total basepairs of peaks: 234727859


In [15]:
with open("peaks.narrowPeak","w") as f:
    peaks.write_to_narrowPeak(f)

## Call peaks for the certain cluster of cells

In this section, we will illustrate how to call peaks for a subset of cells from clustering analysis. Assuming that we have clustering results saved in the file 'analysis/clustering/graphclust/clusters.csv'. We downloaded the results from 10X website.

In [16]:
# Read the CSV from clustering analysis
df = pd.read_csv("./analysis/clustering/graphclust/clusters.csv")

# Get barcodes for each cluster
barcodes_cluster_dict = {}
for c in df["Cluster"].unique():
    df_c = df[df["Cluster"] == c]
    barcodes_cluster_dict[c] = set([x.encode() for x in df_c["Barcode"].tolist()])
    print(f"Number of cell barcodes in cluster {c}: {len(barcodes_cluster_dict[c])}")

Number of cell barcodes in cluster 9: 620
Number of cell barcodes in cluster 3: 813
Number of cell barcodes in cluster 1: 1222
Number of cell barcodes in cluster 7: 639
Number of cell barcodes in cluster 6: 658
Number of cell barcodes in cluster 4: 772
Number of cell barcodes in cluster 2: 997
Number of cell barcodes in cluster 5: 770
Number of cell barcodes in cluster 8: 621
Number of cell barcodes in cluster 12: 428
Number of cell barcodes in cluster 13: 379
Number of cell barcodes in cluster 11: 477
Number of cell barcodes in cluster 10: 537
Number of cell barcodes in cluster 14: 269
Number of cell barcodes in cluster 18: 226
Number of cell barcodes in cluster 15: 262
Number of cell barcodes in cluster 19: 102
Number of cell barcodes in cluster 16: 246
Number of cell barcodes in cluster 17: 235


In the following analysis, we will focus on cluster 1 and cluster 10 -- the biggest cluster and the smallest cluster.

In [17]:
# Take subset of fragments for c1
petrack_c1 = petrack.subset(barcodes_cluster_dict[1])

# Take subset of fragments for c10
petrack_c10 = petrack.subset(barcodes_cluster_dict[10])

Let's get some basic stats of the two subsets of fragments.

In [18]:
print(f"Number of fragments for cluster 1: {petrack_c1.total}")
print(f"Number of fragments for cluster 10: {petrack_c10.total}")

Number of fragments for cluster 1: 35551266
Number of fragments for cluster 10: 16631656


Similar to previous steps, we can generate pileup tracks for these two clusters, make the background tracks, calculate scores, and then call peaks with certain cutoff.

In [19]:
# For cluster 1
pileup_track_c1 = petrack_c1.pileup_bdg()
global_bg_track_c1 = pileup_track_c1.set_single_value(pileup_track_c1.summary()[4]) 
score_track_c1 = pileup_track_c1.make_ScoreTrackII_for_macs(global_bg_track_c1, depth1=100, depth2=100)
score_track_c1.change_score_method(ord('p'))
peaks_c1 = score_track_c1.call_peaks(cutoff=5)

# For cluster 10
pileup_track_c10 = petrack_c10.pileup_bdg()
global_bg_track_c10 = pileup_track_c10.set_single_value(pileup_track_c10.summary()[4]) 
score_track_c10 = pileup_track_c10.make_ScoreTrackII_for_macs(global_bg_track_c10, depth1=100, depth2=100)
score_track_c10.change_score_method(ord('p'))
peaks_c10 = score_track_c10.call_peaks(cutoff=5)


In [20]:
(pileup_sum, pileup_length, pileup_max, pileup_min, pileup_mean, pileup_std) = pileup_track_c1.summary()
print(f"{pileup_sum=}\n{pileup_length=}\n{pileup_max=}\n{pileup_min=}\n{pileup_mean=}\n{pileup_std=}")

pileup_sum=5064676352.0
pileup_length=3086780615
pileup_max=940.0
pileup_min=0.0
pileup_mean=1.640763282775879
pileup_std=12.390028953552246


In [21]:
(pileup_sum, pileup_length, pileup_max, pileup_min, pileup_mean, pileup_std) = pileup_track_c10.summary()
print(f"{pileup_sum=}\n{pileup_length=}\n{pileup_max=}\n{pileup_min=}\n{pileup_mean=}\n{pileup_std=}")

pileup_sum=2456030464.0
pileup_length=3086611397
pileup_max=414.0
pileup_min=0.0
pileup_mean=0.7957044243812561
pileup_std=5.5370354652404785


Let's check some basic facts of the peaks called from only the cluster 1 or cluster 10 cell barcodes.

In [22]:
regions_c1 = Regions()
regions_c1.init_from_PeakIO(peaks_c1)
print(f"Number of peaks of cluster 1: {regions_c1.total}")
print(f"Total basepairs of peaks of cluster 1: {regions_c1.total_length()}")

regions_c10 = Regions()
regions_c10.init_from_PeakIO(peaks_c10)
print(f"Number of peaks of cluster 10: {regions_c10.total}")
print(f"Total basepairs of peaks of cluster 10: {regions_c10.total_length()}")

Number of peaks of cluster 1: 66369
Total basepairs of peaks of cluster 1: 57119357
Number of peaks of cluster 10: 54800
Total basepairs of peaks of cluster 10: 39590493


In [23]:
with open("peaks_c1.narrowPeak","w") as f:
    peaks_c1.write_to_narrowPeak(f)

with open("peaks_c10.narrowPeak","w") as f:
    peaks_c10.write_to_narrowPeak(f)

## Compare cluster-specific peaks

We can try to compare these two sets of peaks using functions provided by `Regions` class. Such as get the overlapping regions...

In [24]:
overlapped_regions_c1_c10 = regions_c1.intersect(regions_c10)

print(f"Number of overlapping regions: {overlapped_regions_c1_c10.total}")
print(f"And total basepairs of overlapping regions: {overlapped_regions_c1_c10.total_length()}")

Number of overlapping regions: 37992
And total basepairs of overlapping regions: 28824472


Or unique regions/peaks of cluster 10 that can not be found in the peaks called from cluster 1.

In [25]:
# make Regions for c10 peaks again since later we will alter this object with .exclude function
unique_regions_c10_vs_c1 = Regions()
unique_regions_c10_vs_c1.init_from_PeakIO(peaks_c10)
# then
unique_regions_c10_vs_c1.exclude(regions_c1)

In [26]:
print(f"Peaks of cluster 10 - cluster 1: {unique_regions_c10_vs_c1.total}")
print(f"And total basepairs in peaks of cluster 10 - cluster 1: {unique_regions_c10_vs_c1.total_length()}")

Peaks of cluster 10 - cluster 1: 17593
And total basepairs in peaks of cluster 10 - cluster 1: 8625919


In [27]:
with open("peaks_c10_vs_c1.bed","w") as f:
    unique_regions_c10_vs_c1.write_to_bed(f)

## Missed rare cluster-specific peaks

Some rare cluster-specific peaks may be missing while we call peaks using all cell barcodes. Therefore, we can compare the cluster-specific peaks with the peaks called from all barcodes.

In [28]:
# make Regions for c10 peaks again since later we will alter this object with .exclude function
unique_regions_c10 = Regions()
unique_regions_c10.init_from_PeakIO(peaks_c10)
# then
unique_regions_c10.exclude(regions)

Yes. We do identify rare peaks for cluster 10.

In [29]:
print(f"Number of rare peaks in cluster 10: {unique_regions_c10.total}")

Number of rare peaks in cluster 10: 1400


In [31]:
with open("unique_peaks_c10.bed","w") as f:
    unique_regions_c10.write_to_bed(f)